In [1]:
import wradlib
import matplotlib.pyplot as plt
import pandas as pd
from osgeo import ogr, osr
import numpy as np
from descartes import PolygonPatch

/home/maik/miniconda3/envs/wradlib/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [3]:
#radars = pd.read_csv(r"P:\progress\global_radar_map\GlobalRadarStations.csv", sep = ";", encoding="cp1252")

In [4]:
radars = pd.read_csv("WRD_radar info.csv", sep = ";", encoding="cp1252")

In [6]:
radars["x"] = radars.Lon_Deg + radars.Lon_Min/60. + radars.Lon_Sec/3600.
radars["y"] = radars.Lat_Deg + radars.Lat_Min/60. + radars.Lat_Sec/3600.
radars.loc[radars.Lon_Dir=="W", "x"] = -radars.loc[radars.Lon_Dir=="W", "x"]
radars.loc[radars.Lat_Dir=="S", "y"] = -radars.loc[radars.Lat_Dir=="S", "y"]

In [7]:
china = pd.read_csv("others/china.csv", sep = ",", encoding="cp1252")
phili = pd.read_csv("others/philippines.csv", sep = ";", encoding="cp1252")
others = pd.read_csv("others/others.csv", sep = ";", encoding="cp1252")

In [8]:
x = np.array([])
y = np.array([])
x = np.append(x, radars.x)
y = np.append(y, radars.y)
x = np.append(x, china.x)
y = np.append(y, china.y)
x = np.append(x, phili.x)
y = np.append(y, phili.y)
x = np.append(x, others.x)
y = np.append(y, others.y)

In [9]:
plt.plot(radars.x, radars.y, "k+")

In [14]:
plt.plot(china.x, china.y, "k+")
#plt.plot(radars[radars.Country=="China"].x, radars[radars.Country=="China"].y, "r+")

In [15]:
# Target projection
#   Robinson
proj = wradlib.georef.epsg_to_osr(54030)
#   Mercator world
#proj = wradlib.georef.proj4_to_osr("+proj=merc +lon_0=0 +k=1 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs") 
#   Winkel Tripel
#proj = wradlib.georef.epsg_to_osr(54018)
#proj = wradlib.georef.wkt_to_osr('PROJCS["World_Winkel_Tripel_NGS",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Winkel_Tripel"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",15.0],PARAMETER["Standard_Parallel_1",40.0],UNIT["Meter",1.0]]')

In [16]:
source = wradlib.georef.get_default_projection()
transform = osr.CoordinateTransformation(source, proj)
retransform = osr.CoordinateTransformation(proj, source)

In [17]:
def write_geojson(fname, geometry, geom_type):
    """Write GeoJSON file for ogr geometries
    
    Parameters
    ----------
    fname : string
    geometry : ogr Geometry object
    geom_type : ogr Geometry type, 
        e.g. ogr.wkbMultiPolygon, ogr.wkbPolygon, ogr.wkbMultiPoint
    
    """
    outDriver = ogr.GetDriverByName('GeoJSON')

    # Create the output GeoJSON
    outDataSource = outDriver.CreateDataSource(fname)
    outLayer = outDataSource.CreateLayer(fname, geom_type=geom_type)

    # Get the output Layer's Feature Definition
    featureDefn = outLayer.GetLayerDefn()

    # create a new feature
    outFeature = ogr.Feature(featureDefn)

    # Set new geometry
    outFeature.SetGeometry(geometry)

    # Add new feature to output Layer
    outLayer.CreateFeature(outFeature)

    # dereference the feature
    outFeature = None

    # Save and close DataSources
    outDataSource = None

In [18]:
r = 200000
phi = np.arange(0, 360, 2)
theta = 0.5
bufrs = []
for i in range(len(x)):
    sitecoords = (x[i], y[i])
    pts = wradlib.georef.polar.spherical_to_proj(
        np.repeat(r, len(phi)), phi, np.repeat(theta, len(phi)), sitecoords
    )
    # Create ring
    ring = ogr.Geometry(ogr.wkbLinearRing)
    for pt in pts:
        ring.AddPoint(pt[0], pt[1])
    # Close ring
    ring.AddPoint(pts[0][0], pts[0][1])

    # Create polygon
    poly = ogr.Geometry(ogr.wkbPolygon)
    poly.AddGeometry(ring)
    bufrs.append(poly)

In [19]:
union = bufrs[0]
for i, bufr in enumerate(bufrs[1:]):
    #if i in [315,]:
    #    continue
    try:
        union = union.Union(bufr)
    except RuntimeError:
        print("Cannot process i=", i)
#union.Transform(transform)

Cannot process i= 179


In [20]:
# Create ring
ring = ogr.Geometry(ogr.wkbLinearRing)
for i in np.linspace(-89.99, 89.99, 100):
    ring.AddPoint(-179.99, i)
for i in np.linspace(-179.99, 179.99, 100):
    ring.AddPoint(i, 89.99)
for i in np.linspace(89.99, -89.99, 100):
    ring.AddPoint(179.99, i)
for i in np.linspace(179.99, -179.99, 100):
    ring.AddPoint(i, -89.99)

# Create polygon
poly = ogr.Geometry(ogr.wkbPolygon)
poly.AddGeometry(ring)
#poly.Transform(transform)

write_geojson("out/clipfrom.geojson", poly, ogr.wkbPolygon)

In [21]:
clipped = poly.Difference(union)
#clipped.Transform(retransform)

In [22]:
write_geojson("out/clipped.geojson", clipped, ogr.wkbMultiPolygon)

In [188]:
#clipped_dict = eval(clipped.ExportToJson())

In [189]:
#fig = plt.figure() 
#ax = fig.gca() 
#ax.add_patch(PolygonPatch(clipped_dict, fc="black", ec="none"))
#ax.axis('scaled')

In [190]:
#dem = wradlib.io.read_generic_netcdf(r"P:\progress\global_radar_map\GMTED2010_15n030_0125deg.nc")
#
#lat = dem["variables"]["latitude"]["data"]
#lon = dem["variables"]["longitude"]["data"]
#elev = dem["variables"]["elevation"]["data"]
#
#lon, lat = np.meshgrid(lon, lat)

In [91]:
#x, y = wradlib.georef.reproject(lon, lat, projection_target=proj)

In [23]:
grdlon, grdlat = np.meshgrid(range(-180, 200, 20), range(-90,92,2))
grdx, grdy = wradlib.georef.reproject(grdlon, grdlat, projection_target=proj)

grdlon, grdlat = np.meshgrid(range(-180, 182, 2), range(-90, 100, 10))
grdx_, grdy_ = wradlib.georef.reproject(grdlon.T, grdlat.T, projection_target=proj)

In [24]:
clipped.Transform(transform)
clipped_dict = eval(clipped.ExportToJson())

In [28]:
ds = wradlib.io.open_raster("/media/x/data/etopo1/DEM_geotiff/alwdgg.tif")#E:\data\etopo1\DEM_geotiff\alwdgg.tif")
# pixel_spacing is in output units (lonlat)
#ds = wradlib.georef.reproject_raster_dataset(ds, spacing=0.1, projection_target=proj)
rastervalues, rastercoords, projsrc = wradlib.georef.extract_raster_dataset(ds)
rastercoords[...,0][rastercoords[...,0] > 180.] = 179.999
rastercoords[...,0][rastercoords[...,0] < -180.] = -179.999
rastercoords[...,1][rastercoords[...,1] > 90.] = 89.999
rastercoords[...,1][rastercoords[...,1] < -90.] = -89.999
# specify kwargs for plotting, using terrain colormap and LogNorm

In [29]:
etopo_x, etopo_y = wradlib.georef.reproject(rastercoords[..., 0], rastercoords[..., 1], projection_target=proj)

In [31]:
dataset, inLayer = wradlib.io.open_vector("/media/x/data/natural_earth/ne_50m_coastline.shp")#(r"E:\data\natural_earth\ne_50m_coastline.shp")
costlines, keys = wradlib.georef.get_vector_coordinates(inLayer, dest_srs=proj)

dataset, inLayer = wradlib.io.open_vector("/media/x/data/natural_earth/ne_50m_admin_0_countries/ne_50m_admin_0_countries.shp")#(r"E:\data\natural_earth\ne_50m_admin_0_countries\ne_50m_admin_0_countries.shp")
borders, keys = wradlib.georef.get_vector_coordinates(inLayer, dest_srs=proj)

/home/maik/miniconda3/envs/wradlib/lib/python3.7/site-packages/wradlib/georef/vector.py:96: UserWarning: geometry without spatial reference - assuming wgs84
  warnings.warn("geometry without spatial reference - assuming wgs84")


In [196]:
#inLayer.SetSpatialFilter(union)
#borders, keys = wradlib.georef.get_vector_coordinates(inLayer, dest_srs=proj)

In [32]:
from matplotlib import colors

bounds=[-10000, -5000, -4000, -2000, -1000, -500, -100, -10, 0, 10, 50, 100, 200, 300, 400, 500, 600, 700, 1000, 1500, 2000, 10000]
cmap = colors.ListedColormap(plt.cm.gist_earth(np.linspace(0,1,len(bounds)-1))) #colors.ListedColormap(['white', 'red'])
norm = colors.BoundaryNorm(bounds, cmap.N)

fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(111, aspect="equal")
fig.patch.set_visible(False)
ax.axis('off')
img = plt.pcolormesh(etopo_x, etopo_y, rastervalues, cmap=cmap, norm=norm)#cmap=plt.cm.terrain)
#cb = plt.colorbar(img, cmap=cmap, norm=norm, boundaries=bounds, ticks=bounds)
ax.add_patch(PolygonPatch(clipped_dict, fc="black", ec="black", alpha=0.7))
wradlib.vis.add_lines(ax, borders, color='black', lw=0.3)
latlines = plt.plot(grdx, grdy, "w-", linewidth=0.2) 
lonlines = plt.plot(grdx_, grdy_, "w-", linewidth=0.2)

plt.tight_layout()

#plt.savefig("globalmap.png", dpi=300)

In [33]:
# ... Stack them
rgb = np.dstack([rastervalues[0], rastervalues[1], rastervalues[2]])

# Reshape as list of rgb tuples
color_tuples = np.array([rastervalues[0].flatten(), rastervalues[1].flatten(), rastervalues[2].flatten()]).transpose()

In [34]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, aspect='equal')
dem = ax.pcolormesh(rastercoords[..., 0], rastercoords[..., 1],
                    rastervalues, cmap=plt.cm.terrain)

In [ ]:
plt.pcolormesh(rastervalues[0], color=color_tuples)